 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/rnn_test.ipynb#Set-up-data" data-toc-modified-id="Set-up-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set up data</a></span></li><li><span><a href="http://localhost:8888/notebooks/rnn_test.ipynb#Build-model" data-toc-modified-id="Build-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Build model</a></span></li></ul></div>

In [47]:
# Imports
from importlib import reload
from crps_loss import crps_cost_function
import utils; reload(utils)
from utils import *
import matplotlib.pyplot as plt
%matplotlib inline

import keras
from keras.layers import Input, Dense, merge, Embedding, Flatten, Dropout, \
    SimpleRNN, LSTM, TimeDistributed
from keras.layers.merge import Concatenate
from keras.models import Model, Sequential
import keras.backend as K
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, Adam

In [2]:
# Basic setup
DATA_DIR = '/Volumes/STICK/data/ppnn_data/'  # Mac
# DATA_DIR = '/project/meteo/w2w/C7/ppnn_data/'   # LMU
results_dir = '../results/'
window_size = 25   # Days in rolling window
fclt = 48   # Forecast lead time in hours

## Set up data

In [3]:

train_dates = ['2015-01-01', '2016-01-01']
test_dates =  ['2016-01-01', '2017-01-01']
train_set, test_set = get_train_test_sets(DATA_DIR, train_dates, test_dates)

train set contains 365 days
test set contains 366 days


In [4]:
train_set.features.shape

(180849, 2)

In [8]:
targets, features, dates, station_id, feature_names = load_raw_data(DATA_DIR)

In [13]:
targets.shape, features.shape

((3653, 537), (2, 3653, 537))

In [11]:
train_dates_idxs = [return_date_idx(dates, train_dates[0]),
                            return_date_idx(dates, train_dates[1])]
test_dates_idxs = [return_date_idx(dates, test_dates[0]),
                           return_date_idx(dates, test_dates[1])]

In [12]:
train_dates_idxs

[2920, 3285]

In [14]:
x_train = features[:, train_dates_idxs[0]:train_dates_idxs[1]]
y_train = targets[train_dates_idxs[0]:train_dates_idxs[1]]

In [15]:
x_train.shape

(2, 365, 537)

In [16]:
# Reshape them
seq_len = 2

In [24]:
x_rnn = np.stack([x_train[:, 1:, :], x_train[:, :-1, :]])

In [25]:
x_rnn.shape # [seq, feature, day, station]

(2, 2, 364, 537)

In [26]:
x_rnn[:, 1, 0, 0]

array([ 0.60977471,  1.36875927], dtype=float32)

In [31]:
np.isnan(y_train).sum()

0

In [32]:
y_rnn = np.stack([y_train[1:, :], y_train[:-1, :]])
y_rnn.shape

(2, 364, 537)

In [33]:
# Ravel arrays
x_rnn = np.reshape(x_rnn, (2, 2, -1))
y_rnn = np.reshape(y_rnn, (2, -1))
x_rnn.shape, y_rnn.shape # [seq, feature, instance]

((2, 2, 195468), (2, 195468))

In [36]:
x_rnn = np.rollaxis(x_rnn, 2, 0)
y_rnn = np.rollaxis(y_rnn, 1, 0)
x_rnn.shape, y_rnn.shape # [sample, time step feature]

((195468, 2, 2), (195468, 2))

In [68]:
y_rnn = np.atleast_3d(y_rnn)
y_rnn.shape

(195468, 2, 1)

## Build model

In [69]:
batch_size = 64
hidden_nodes = 10

In [74]:
inp = Input(shape=(2, 2, ))
x = LSTM(hidden_nodes, return_sequences=True)(inp)
x = TimeDistributed(Dense(1, activation='linear'))(x)
rnn_model = Model(inputs=inp, outputs=x)

INFO (theano.gof.compilelock): Refreshing lock /Users/stephanrasp/.theano/compiledir_Darwin-17.0.0-x86_64-i386-64bit-i386-3.6.2-64/lock_dir/lock


In [75]:
rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 2, 2)              0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 2, 10)             520       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 2, 1)              11        
Total params: 531
Trainable params: 531
Non-trainable params: 0
_________________________________________________________________


In [76]:
rnn_model.compile(optimizer=Adam(0.01), loss='mse')

In [77]:
rnn_model.fit(x_rnn, y_rnn, epochs=1, batch_size=batch_size)

Epoch 1/1
195468/195468 [==============================] - 7s - loss: nan     
